In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
RS = sum(list(map(ord, 'Dale Boca')))

import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

import numpy as np
import os;

### Preparación de datasets

In [2]:
from sklearn.preprocessing import LabelEncoder

def dataset_prep(window=3, neutral=True):
    
    train = pd.read_csv('../data/trainset_va.csv', index_col=0)
    test  = pd.read_csv('../data/testset_va.csv', index_col=0)
    
    train = pd.concat([train, test.iloc[:92]])
    test  = test.iloc[92:]
    
    cols1 = ['pos', 'neg', 'neu'] if neutral==True else ['pos', 'neg']
    
    for col in cols1:
        train[col+'w'] = train[col].rolling(window=window).mean()#sum()
        test[col+'w'] = test[col].rolling(window=window).mean()#sum()
    
    train['exc_ret'] = train['exc_ret'].shift(periods=-1)
    test['exc_ret']  = test['exc_ret'].shift(periods=-1)
    
    train.dropna(axis=0, inplace=True)
    test.dropna(axis=0, inplace=True)
    
    cols2 = [x+'w' for x in cols1]
    
    X = train[cols2]
    y = train.exc_ret
    Xt = test[cols2]
    yt = test.exc_ret
    
    le = LabelEncoder()
    y  = le.fit_transform(y)
    yt = le.fit_transform(yt)
    
    return X, Xt, y, yt    

### Entrenamiento de Clasificadores

In [3]:
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV, KFold, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from scikitplot.metrics import plot_roc

from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.svm import SVC

In [4]:
def train_classifier(X, Xt, y, yt):
    
    #model_name = ['Random Forest', 'XGBoost', 'LightGBM', 'Logistic Regression',
    #              'Catboost', 'Support Vector']
    model_name = ['Random Forest', 'XGBoost', 'LightGBM', 'Catboost']
   
    model_init = [RandomForestClassifier(),
                  XGBClassifier(),
                  lgb.LGBMClassifier(),
                  #LogisticRegression(),
                  CatBoostClassifier(),
                  #SVC(probability=True)
                 ]
    
    # Random Forest 
    params1 = {  
        "n_estimators": st.randint(10,100),    # Number of boosted trees to fit.
        "max_depth"   : st.randint(2, 25),     # Maximum tree depth for base learners.
    }
    
    # XGBoost y Lightgbm
    params2 = {  
        "n_estimators": st.randint(10,100),    # Number of boosted trees to fit.
        "max_depth": st.randint(2, 25),         # Maximum tree depth for base learners.
        "learning_rate": st.uniform(0.01, 0.5), # Boosting learning rate (xgb’s “eta”)
        "colsample_bytree": st.beta(10, 1),     # Subsample ratio of columns when constructing each tree.
        "subsample": st.beta(10, 1),            # Subsample ratio of the training instance.
        "gamma": st.uniform(0, 10),             # Minimum loss reduction required to make a further partition on a leaf node of the tree.
        'reg_alpha': st.uniform(0.05,10),       # L1 regularization term on weights
        "min_child_weight": st.uniform(1,20),   # Minimum sum of instance weight(hessian) needed in a child.
    }
    
    # Logistic Regression
    params3 = {  
        "penalty"     : ['l1', 'l2'],
        "C"           : st.uniform(0.1, 10.),
    }
    
    # Catboost
    params4 = {}
    
    # SVC
    params5 = {
        "C"           : st.uniform(0.1, 10.),
        #"kernel"      : ['lbf', 'linear']
    }
    
    #model_params = [params1, params2, params2, params3, params4, params5]
    model_params = [params1, params2, params2, params4]
    
    train_scores    = []
    test_scores     = []
    best_estimators = []
    best_parameters = []
    
    for name, mod, params in zip(model_name, model_init, model_params):
        model = mod
        #kf    = KFold(n_splits=5, shuffle=True, random_state=RS)
        tsp   = TimeSeriesSplit(n_splits=8)
        rgrid = RandomizedSearchCV(estimator=model, param_distributions=params, cv=tsp,
                                   scoring='roc_auc', n_iter=50, verbose=1, n_jobs=-1)
        rgrid.fit(X, y)
        best_estimators.append(rgrid.best_estimator_)
        best_parameters.append(rgrid.best_params_)
        train_scores.append(rgrid.best_score_)
        ppred = rgrid.predict_proba(Xt)
        score = roc_auc_score(y_score=ppred[:, 1], y_true=yt)
        test_scores.append(score)
    
    results = pd.DataFrame(
        {'Model'      : model_name,
         'Train Score': train_scores,
         'Test Score' : test_scores,
         'Params'     : best_parameters,
         'Estimator'  : best_estimators
        })
    
    return results

In [5]:
import itertools
windows = np.arange(1, 6)
neutral = [1, 0]

resultado = pd.DataFrame()

for win, neu in itertools.product(windows, neutral):
    
    X, X_test, y, y_test = dataset_prep(window=win, neutral=neu)
    
    clfs = train_classifier(X, X_test, y, y_test)
  
    res = pd.DataFrame(
    {'Model'      : clfs['Model'],
     'Window'     : list(str(win)*clfs.shape[0]),
     'Neutral'    : list(str(neu)*clfs.shape[0]),
     'Train Score': clfs['Train Score'],
     'Test Score' : clfs['Test Score'],
     'Params'     : clfs['Params'],
     'Estimator'  : clfs['Estimator']
    })
    
    resultado = pd.concat([resultado, res], axis=0)

Fitting 8 folds for each of 50 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:   12.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 8 folds for each of 50 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 8 folds for each of 50 candidates, totalling 400 fits


...

168:	learn: 0.6113094	total: 1.42s	remaining: 7.01s
169:	learn: 0.6110843	total: 1.43s	remaining: 6.99s
170:	learn: 0.6106164	total: 1.44s	remaining: 6.98s
171:	learn: 0.6104579	total: 1.45s	remaining: 6.96s
172:	learn: 0.6102035	total: 1.45s	remaining: 6.95s
173:	learn: 0.6101023	total: 1.46s	remaining: 6.94s
174:	learn: 0.6100408	total: 1.47s	remaining: 6.92s
175:	learn: 0.6096907	total: 1.48s	remaining: 6.91s
176:	learn: 0.6093955	total: 1.48s	remaining: 6.89s
177:	learn: 0.6091630	total: 1.49s	remaining: 6.88s
178:	learn: 0.6087267	total: 1.5s	remaining: 6.87s
179:	learn: 0.6081446	total: 1.5s	remaining: 6.85s
180:	learn: 0.6079298	total: 1.51s	remaining: 6.84s
181:	learn: 0.6076132	total: 1.52s	remaining: 6.83s
182:	learn: 0.6073859	total: 1.53s	remaining: 6.82s
183:	learn: 0.6071064	total: 1.53s	remaining: 6.81s
184:	learn: 0.6068958	total: 1.54s	remaining: 6.79s
185:	learn: 0.6065788	total: 1.55s	remaining: 6.78s
186:	learn: 0.6064487	total: 1.56s	remaining: 6.76s
187:	learn: 0.

347:	learn: 0.5709714	total: 2.79s	remaining: 5.23s
348:	learn: 0.5708402	total: 2.8s	remaining: 5.22s
349:	learn: 0.5706561	total: 2.81s	remaining: 5.21s
350:	learn: 0.5706418	total: 2.81s	remaining: 5.2s
351:	learn: 0.5704381	total: 2.82s	remaining: 5.19s
352:	learn: 0.5703853	total: 2.83s	remaining: 5.18s
353:	learn: 0.5702242	total: 2.83s	remaining: 5.17s
354:	learn: 0.5699577	total: 2.84s	remaining: 5.16s
355:	learn: 0.5699065	total: 2.85s	remaining: 5.15s
356:	learn: 0.5698075	total: 2.85s	remaining: 5.14s
357:	learn: 0.5695378	total: 2.86s	remaining: 5.13s
358:	learn: 0.5694315	total: 2.87s	remaining: 5.12s
359:	learn: 0.5693147	total: 2.88s	remaining: 5.11s
360:	learn: 0.5691611	total: 2.88s	remaining: 5.11s
361:	learn: 0.5688476	total: 2.89s	remaining: 5.09s
362:	learn: 0.5687596	total: 2.9s	remaining: 5.09s
363:	learn: 0.5687144	total: 2.9s	remaining: 5.08s
364:	learn: 0.5685460	total: 2.92s	remaining: 5.08s
365:	learn: 0.5684776	total: 2.93s	remaining: 5.08s
366:	learn: 0.56

507:	learn: 0.5456743	total: 3.97s	remaining: 3.84s
508:	learn: 0.5456395	total: 3.98s	remaining: 3.83s
509:	learn: 0.5455004	total: 3.98s	remaining: 3.83s
510:	learn: 0.5453449	total: 3.99s	remaining: 3.82s
511:	learn: 0.5452232	total: 4s	remaining: 3.81s
512:	learn: 0.5451271	total: 4s	remaining: 3.8s
513:	learn: 0.5451039	total: 4.01s	remaining: 3.79s
514:	learn: 0.5448390	total: 4.02s	remaining: 3.78s
515:	learn: 0.5444694	total: 4.03s	remaining: 3.78s
516:	learn: 0.5443840	total: 4.03s	remaining: 3.77s
517:	learn: 0.5443502	total: 4.04s	remaining: 3.76s
518:	learn: 0.5441712	total: 4.05s	remaining: 3.75s
519:	learn: 0.5440176	total: 4.06s	remaining: 3.74s
520:	learn: 0.5440012	total: 4.06s	remaining: 3.73s
521:	learn: 0.5438142	total: 4.07s	remaining: 3.73s
522:	learn: 0.5437937	total: 4.08s	remaining: 3.72s
523:	learn: 0.5437203	total: 4.08s	remaining: 3.71s
524:	learn: 0.5435128	total: 4.09s	remaining: 3.7s
525:	learn: 0.5435070	total: 4.09s	remaining: 3.69s
526:	learn: 0.543471

689:	learn: 0.5259272	total: 5.34s	remaining: 2.4s
690:	learn: 0.5256525	total: 5.35s	remaining: 2.39s
691:	learn: 0.5256248	total: 5.36s	remaining: 2.38s
692:	learn: 0.5255746	total: 5.37s	remaining: 2.38s
693:	learn: 0.5254693	total: 5.38s	remaining: 2.37s
694:	learn: 0.5253910	total: 5.38s	remaining: 2.36s
695:	learn: 0.5253531	total: 5.39s	remaining: 2.35s
696:	learn: 0.5250719	total: 5.4s	remaining: 2.35s
697:	learn: 0.5250099	total: 5.4s	remaining: 2.34s
698:	learn: 0.5249810	total: 5.41s	remaining: 2.33s
699:	learn: 0.5247645	total: 5.42s	remaining: 2.32s
700:	learn: 0.5247040	total: 5.42s	remaining: 2.31s
701:	learn: 0.5246564	total: 5.43s	remaining: 2.31s
702:	learn: 0.5245862	total: 5.44s	remaining: 2.3s
703:	learn: 0.5244874	total: 5.45s	remaining: 2.29s
704:	learn: 0.5244441	total: 5.45s	remaining: 2.28s
705:	learn: 0.5240170	total: 5.46s	remaining: 2.27s
706:	learn: 0.5238987	total: 5.47s	remaining: 2.27s
707:	learn: 0.5238888	total: 5.47s	remaining: 2.26s
708:	learn: 0.52

853:	learn: 0.5086974	total: 6.5s	remaining: 1.11s
854:	learn: 0.5085730	total: 6.51s	remaining: 1.1s
855:	learn: 0.5085039	total: 6.52s	remaining: 1.1s
856:	learn: 0.5084669	total: 6.52s	remaining: 1.09s
857:	learn: 0.5083546	total: 6.53s	remaining: 1.08s
858:	learn: 0.5083194	total: 6.54s	remaining: 1.07s
859:	learn: 0.5082738	total: 6.54s	remaining: 1.06s
860:	learn: 0.5082729	total: 6.55s	remaining: 1.06s
861:	learn: 0.5082308	total: 6.56s	remaining: 1.05s
862:	learn: 0.5081435	total: 6.57s	remaining: 1.04s
863:	learn: 0.5081329	total: 6.57s	remaining: 1.03s
864:	learn: 0.5081011	total: 6.58s	remaining: 1.03s
865:	learn: 0.5078926	total: 6.58s	remaining: 1.02s
866:	learn: 0.5078466	total: 6.59s	remaining: 1.01s
867:	learn: 0.5077612	total: 6.6s	remaining: 1s
868:	learn: 0.5075790	total: 6.61s	remaining: 996ms
869:	learn: 0.5073853	total: 6.61s	remaining: 988ms
870:	learn: 0.5073162	total: 6.62s	remaining: 981ms
871:	learn: 0.5070582	total: 6.63s	remaining: 973ms
872:	learn: 0.50688

In [9]:
resultado.sort_values(by='Train Score', ascending=False)#.to_csv('resultados/res_preliminar_4_TW.csv')

Model Window Neutral  Train Score  Test Score  \
3  Catboost       4      0       0.561156     0.602564     
2  LightGBM       2      0       0.560655     0.543269     
2  LightGBM       5      0       0.559450     0.611111     
2  LightGBM       4      0       0.555517     0.589744     
1  XGBoost        4      0       0.555295     0.551282     
2  LightGBM       4      1       0.547800     0.628205     
0  Random Forest  4      0       0.547041     0.589744     
3  Catboost       4      1       0.546528     0.576923     
0  Random Forest  4      1       0.544991     0.487179     
0  Random Forest  2      0       0.544112     0.634615     
1  XGBoost        4      1       0.543781     0.500000     
2  LightGBM       2      1       0.541067     0.572115     
2  LightGBM       1      1       0.541001     0.437500     
1  XGBoost        2      0       0.540856     0.596154     
2  LightGBM       1      0       0.540229     0.437500     
0  Random Forest  2      1       0.539835     0.528846     
1  XGBoost        5      1       0.536713     0.402778     
1  XGBoost        1      0       0.535831     0.428571     
2  LightGBM       5      1       0.532642     0.458333     
2  LightGBM       3      1       0.532220     0.450549     
1  XGBoost        2      1       0.526510     0.528846     
2  LightGBM       3      0       0.524110     0.505495     
0  Random Forest  1      1       0.522795     0.446429     
1  XGBoost        5      0       0.521889     0.618056     
3  Catboost       2      0       0.521740     0.692308     
3  Catboost       2      1       0.519806     0.625000     
1  XGBoost        3      1       0.518688     0.489011     
0  Random Forest  5      0       0.518434     0.569444     
1  XGBoost        3      0       0.513570     0.489011     
1  XGBoost        1      1       0.509051     0.500000     
0  Random Forest  1      0       0.505127     0.500000     
0  Random Forest  5      1       0.500161     0.500000     
0  Random Forest  3      1       0.488646     0.500000     
0  Random Forest  3      0       0.486968     0.483516     
3  Catboost       5      1       0.484662     0.458333     
3  Catboost       5      0       0.463828     0.611111     
3  Catboost       1      0       0.462585     0.446429     
3  Catboost       3      0       0.456648     0.472527     
3  Catboost       1      1       0.451330     0.491071     
3  Catboost       3      1       0.436558     0.241758     

                                                                                                                                                                                                                                                       Params  \
3  {}                                                                                                                                                                                                                                                           
2  {'colsample_bytree': 0.9897391773757975, 'gamma': 2.5618131929675823, 'learning_rate': 0.3500739813681228, 'max_depth': 3, 'min_child_weight': 3.7750546325660497, 'n_estimators': 50, 'reg_alpha': 9.985328144238391, 'subsample': 0.9163026661274429}      
2  {'colsample_bytree': 0.9205497104893088, 'gamma': 1.6889390683564542, 'learning_rate': 0.14107848788387012, 'max_depth': 19, 'min_child_weight': 8.304604490146996, 'n_estimators': 90, 'reg_alpha': 1.1121056955495179, 'subsample': 0.8958464240430275}    
2  {'colsample_bytree': 0.9617918043619508, 'gamma': 4.402150892393446, 'learning_rate': 0.08451540635190728, 'max_depth': 10, 'min_child_weight': 6.49739679438408, 'n_estimators': 80, 'reg_alpha': 1.9569159760388433, 'subsample': 0.9777161496705093}      
1  {'colsample_bytree': 0.8844308030103815, 'gamma': 0.5588497771144141, 'learning_rate': 0.507477633382161, 'max_depth': 18, 'min_child_weight': 1.168017283870443, 'n_estimators': 44, 'reg_alpha': 2.0264520227526925, 'subsample': 0.9982156333628359}      
2  {'col